In [1]:
! git clone https://github.com/google-research/tapas.git


Cloning into 'tapas'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 474 (delta 1), reused 7 (delta 1), pack-reused 442
Receiving objects: 100% (474/474), 383.69 KiB | 8.72 MiB/s, done.
Resolving deltas: 100% (277/277), done.


In [2]:
! pip install ./tapas


Processing ./tapas
     |████████████████████████████████| 3.5MB 4.4MB/s 
     |████████████████████████████████| 10.1MB 36.8MB/s 
     |████████████████████████████████| 516.2MB 29kB/s 
     |████████████████████████████████| 716kB 34.3MB/s 
     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 3.5MB 41.1MB/s 
     |████████████████████████████████| 358kB 46.7MB/s 
     |████████████████████████████████| 1.4MB 47.4MB/s 
     |████████████████████████████████| 153kB 39.6MB/s 
     |████████████████████████████████| 81kB 6.7MB/s 
     |████████████████████████████████| 1.2MB 36.3MB/s 
     |████████████████████████████████| 225kB 47.3MB/s 
     |████████████████████████████████| 63.1MB 62kB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 184kB 52.6MB/s 
     |████████████████████████████████| 215kB 50.1MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
     |████████████████████████████████

In [3]:
! gsutil cp gs://tapas_models/2020_04_21/tapas_sqa_base.zip . && unzip tapas_sqa_base.zip


Copying gs://tapas_models/2020_04_21/tapas_sqa_base.zip...
| [1 files][  1.0 GiB/  1.0 GiB]   49.7 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      
Archive:  tapas_sqa_base.zip
   creating: tapas_sqa_base/
  inflating: tapas_sqa_base/model.ckpt.data-00000-of-00001  
  inflating: tapas_sqa_base/model.ckpt.index  
  inflating: tapas_sqa_base/README.txt  
  inflating: tapas_sqa_base/vocab.txt  
  inflating: tapas_sqa_base/bert_config.json  
  inflating: tapas_sqa_base/model.ckpt.meta  


In [3]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

In [4]:

from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

In [5]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [6]:
df = pd.read_csv("Employee Salaries.csv")


In [7]:
df = df.astype(str)


In [8]:
df


,Employee ID,Employee Name,Salary,Designation,State
0,1,John Snow,"$30,000",Account Executive,Maryland
1,2,Paul Das,"$50,000",Sr. Manager,Indiana
2,3,Ranjeet Kumar,"$100,000",VP of Sales,California
3,4,Rahul Dsouza,"$20,000",Business Development Representative,Arkansas
4,5,Greg Patterson,"$45,000",Team Lead,Idaho


In [9]:
list_of_list = [[]]
list_of_list[0] = list(df.columns)
list_of_list.extend(df.values.tolist())

In [10]:
list_of_list

[['Employee ID', 'Employee Name', 'Salary', 'Designation', 'State'],
 ['1', 'John Snow', '$30,000', 'Account Executive', 'Maryland'],
 ['2', 'Paul Das', '$50,000', 'Sr. Manager', 'Indiana'],
 ['3', 'Ranjeet Kumar', '$100,000', 'VP of Sales', 'California'],
 ['4',
  'Rahul Dsouza',
  '$20,000',
  'Business Development Representative',
  'Arkansas'],
 ['5', 'Greg Patterson', '$45,000', 'Team Lead', 'Idaho']]

In [11]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = table_data
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python tapas/tapas/run_task_main.py \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

In [14]:
result = predict(list_of_list, [ "What is the Average Salary of the employees?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Employee ID,Employee Name,Salary,Designation,State
1,John Snow,"$30,000",Account Executive,Maryland
2,Paul Das,"$50,000",Sr. Manager,Indiana
3,Ranjeet Kumar,"$100,000",VP of Sales,California
4,Rahul Dsouza,"$20,000",Business Development Representative,Arkansas
5,Greg Patterson,"$45,000",Team Lead,Idaho



> What is the Average Salary of the employees?
$50,000, $20,000, $100,000, $45,000, $30,000


In [15]:
r = predict(list_of_list, ["Which employee has the Highest Salary?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Employee ID,Employee Name,Salary,Designation,State
1,John Snow,"$30,000",Account Executive,Maryland
2,Paul Das,"$50,000",Sr. Manager,Indiana
3,Ranjeet Kumar,"$100,000",VP of Sales,California
4,Rahul Dsouza,"$20,000",Business Development Representative,Arkansas
5,Greg Patterson,"$45,000",Team Lead,Idaho



> Which employee has the Highest Salary?
Ranjeet Kumar
